In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import image
%matplotlib inline

import glob
import pathlib

import tensorflow as tf
import tensorflow as keras 
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense

In [2]:
img = pathlib.Path('D:\Work - Data\Projects\Red Blood Cells\Images\Mendeley_Data_Barcelona')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# NON UTILISE
# import cv2
# Load and preprocess the image using OpenCV
image = cv2.imread("image.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Apply contrast adjustment
contrast_factor = 1.2  # Example: 20% increase in contrast
adjusted_image = cv2.convertScaleAbs(image, alpha=contrast_factor)

In [17]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Number of classes
num_classes = 8

# Load the pre-trained DenseNet121 model without the top classification layer
# input shape = 225, 225, 3 instead of default 224, 224, 3
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(360, 360, 3)) #(224,224,3)

# Add a global average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer with 256 units and ReLU activation
x = Dense(256, activation='relu')(x)

# Add the output layer with softmax activation for classification
predictions = Dense(num_classes, activation='softmax')(x)

# Create the transfer learning model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the weights of the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with appropriate loss function, optimizer, and metrics
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Define the checkpoint saving callback
checkpoint_path = "best_model_checkpoint.h5"
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

# Define the early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Print the summary of the model
model.summary()



Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 360, 360, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_2 (ZeroPadding2  (None, 366, 366, 3)  0          ['input_2[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1/conv (Conv2D)            (None, 180, 180, 64  9408        ['zero_padding2d_2[0][0]']       
                                )                                                           

In [ ]:
## NOT USED
# Example code for training, modify it based on your dataset and requirements
train_images = ...
train_labels = ...
val_images = ...
val_labels = ...

In [4]:
# NOT USED 
# #removal of test dataset creation, definition of train and validation dataset using Barcelona dataset

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from sklearn.metrics import confusion_matrix
import numpy as np

# set up the data generators
data_dir = 'D:\Work - Data\Projects\Red Blood Cells\Images\Mendeley_Data_Barcelona'
#validation_dir = 'D:\Work - Data\Projects\Red Blood Cells\Images\Mendeley_Data_Barcelona'
#test_dir = 'D:\Work - Data\Projects\Red Blood Cells\Images\Tests\'
img_height = 360
img_width = 360
batch_size = 32

train_data = tf.keras.preprocessing.image_dataset_from_directory(
                                            data_dir, validation_split=0.2, subset='training', seed=123,
                                            image_size=(img_height, img_width), batch_size=batch_size, class_mode='categorical')

validation_data = tf.keras.preprocessing.image_dataset_from_directory(
                                            data_dir, validation_split=0.2, subset='validation', seed=123,
                                            image_size=(img_height, img_width), batch_size=batch_size, class_mode='categorical')

# modification du préprocessing et de la data augmentation avec\
# annulation du shear range qui enlèverait des features légitimes à l'apprentissage, ajout des éléments
# ajout des modifications de la luminosité
# ajout du vertical flip dans l'objectif d'augmenter la généralisation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],  # 20% variation in brightness
)

'''train_data = train_datagen.flow_from_directory(train_dir, target_size=(img_height, img_width), batch_size=batch_size,\
    class_mode='categorical')
validation_data = test_datagen.flow_from_directory(validation_dir, target_size=(img_height, img_width), batch_size=batch_size,\
    class_mode='categorical')'''

Found 15793 images belonging to 8 classes.


NameError: name 'test_datagen' is not defined

In [18]:
from tensorflow.keras.applications.densenet import preprocess_input

data_dir = 'D:\Work - Data\Projects\Red Blood Cells\Images\Mendeley_Data_Barcelona'
img_height = 360
img_width = 360
batch_size = 32

train_data = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir, validation_split=0.2, subset='training', seed=123,
    image_size=(img_height, img_width), batch_size=batch_size,
#    class_mode='categorical'
)

validation_data = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir, validation_split=0.2, subset='validation', seed=123,
    image_size=(img_height, img_width), batch_size=batch_size,
#    class_mode='categorical'
)

train_data = train_data.map(lambda x, y: (preprocess_input(x), y))
train_data = train_data.map(lambda x, y: (tf.image.random_flip_left_right(x), y))
train_data = train_data.map(lambda x, y: (tf.image.random_flip_up_down(x), y))
#train_data = train_data.map(lambda x, y: (tf.image.random_zoom(x, (0.8, 1.2)), y))
#train_data = train_data.map(lambda x, y: (tf.image.random_brightness(x, (0.8, 1.2)), y))
train_data = train_data.map(lambda x, y: (tf.image.random_brightness(x, 0.2), y))
#train_data = train_data.map(lambda x, y: (tf.image.random_contrast(x, 0.8, 1.2), y))
train_data = train_data.map(lambda x, y: (tf.image.random_contrast(x, 0.2, 0.8), y))
#train_data = train_data.map(lambda x, y: (tf.image.random_contrast(x, 0.2), y))
validation_data = validation_data.map(lambda x, y: (preprocess_input(x), y))

#https://www.tensorflow.org/api_docs/python/tf/keras/layers/RandomZoom

'''input_img = np.random.random((32, 224, 224, 3))
layer = tf.keras.layers.RandomZoom(.5, .2)
out_img = layer(input_img)
out_img.shape'''



Found 15793 files belonging to 8 classes.
Using 12635 files for training.
Found 15793 files belonging to 8 classes.
Using 3158 files for validation.


'input_img = np.random.random((32, 224, 224, 3))\nlayer = tf.keras.layers.RandomZoom(.5, .2)\nout_img = layer(input_img)\nout_img.shape'

In [19]:
# Train the model with your dataset and appropriate data preprocessing steps
# Make sure to load your dataset and perform data preprocessing

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

model.fit(
    train_data,
    validation_data=validation_data,#(val_images, val_labels),
    epochs=12,
    batch_size=32,
    callbacks=[checkpoint, early_stop]
)

# Load the best model saved during training
best_model = Model(inputs=base_model.input, outputs=predictions)
best_model.load_weights(checkpoint_path)
# set up the callbacks
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "_" + model_name
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(filepath=f'{model_name}_tunedDN121_best.h5', monitor='val_loss', save_best_only=True)
tensorboard = TensorBoard(log_dir=log_dir)


model.fit(
    train_data,
    validation_data=validation_data,#(val_images, val_labels),
    epochs=12,
    batch_size=32,
    callbacks=[checkpoint, early_stop]
)


Epoch 1/12
395/395 [==============================] - 1917s 5s/step - loss: 0.7497 - acc: 0.7530 - val_loss: 0.5022 - val_acc: 0.8515
Epoch 2/12
395/395 [==============================] - 1976s 5s/step - loss: 0.3744 - acc: 0.8769 - val_loss: 0.4746 - val_acc: 0.8372
Epoch 3/12
395/395 [==============================] - 1988s 5s/step - loss: 0.3093 - acc: 0.8982 - val_loss: 0.3566 - val_acc: 0.8870
Epoch 4/12
395/395 [==============================] - 2076s 5s/step - loss: 0.2694 - acc: 0.9113 - val_loss: 0.3646 - val_acc: 0.8787
Epoch 5/12
395/395 [==============================] - 3773s 10s/step - loss: 0.2462 - acc: 0.9152 - val_loss: 0.3577 - val_acc: 0.8794
Epoch 6/12
395/395 [==============================] - 5706s 14s/step - loss: 0.2454 - acc: 0.9164 - val_loss: 0.2764 - val_acc: 0.9094
Epoch 7/12
395/395 [==============================] - 2075s 5s/step - loss: 0.2223 - acc: 0.9244 - val_loss: 0.2492 - val_acc: 0.9202
Epoch 8/12
395/395 [==============================] - 1869s 

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'best_model_checkpoint.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [1]:
  
# Evaluate the best model on the test set
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from sklearn.metrics import confusion_matrix
import numpy as np

# set up the data generators
test_dir_1 = 'D:\Work - Data\Projects\Red Blood Cells\Images\Tests\TestA'
img_height = 360
img_width = 360
batch_size = 32

test_datagen = ImageDataGenerator( # rescaling without data augmentation
    rescale=1./255
)
test_data_1 = test_datagen.flow_from_directory(test_dir_1,target_size=(img_height, img_width), batch_size=batch_size,\
    class_mode='categorical', shuffle=False)

best_model.evaluate(test_images, test_labels)


Found 4350 images belonging to 8 classes.


NameError: name 'best_model' is not defined

In [31]:

# Train the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_data,
                    epochs=12,
                    validation_data=validation_data,
                    callbacks=[early_stop, model_checkpoint, tensorboard])

# Save the final model
model.save(f'{model_name}.h5')
print(f"{model_name} saved successfully")

# plot the loss and accuracy curves
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()


Epoch 1/12
494/494 [==============================] - 10634s 22s/step - loss: 2.0932 - accuracy: 0.2091 - val_loss: 2.0123 - val_accuracy: 0.2108
Epoch 2/12
494/494 [==============================] - 8973s 18s/step - loss: 2.0142 - accuracy: 0.2091 - val_loss: 2.0111 - val_accuracy: 0.2108
Epoch 3/12
494/494 [==============================] - 8543s 17s/step - loss: 2.0135 - accuracy: 0.2091 - val_loss: 2.0113 - val_accuracy: 0.2108
Epoch 4/12
494/494 [==============================] - 8376s 17s/step - loss: 2.0135 - accuracy: 0.2100 - val_loss: 2.0111 - val_accuracy: 0.2108
Epoch 5/12
494/494 [==============================] - 8080s 16s/step - loss: 2.0129 - accuracy: 0.2108 - val_loss: 2.0109 - val_accuracy: 0.2108
Epoch 6/12
494/494 [==============================] - 8394s 17s/step - loss: 2.0127 - accuracy: 0.2107 - val_loss: 2.0110 - val_accuracy: 0.2108
Epoch 7/12
494/494 [==============================] - ETA: 0s - loss: 2.0120 - accuracy: 0.2107

KeyboardInterrupt: 

Epoch 1/12
494/494 [==============================] - 10634s 22s/step - loss: 2.0932 - accuracy: 0.2091 - val_loss: 2.0123 - val_accuracy: 0.2108
Epoch 2/12
494/494 [==============================] - 8973s 18s/step - loss: 2.0142 - accuracy: 0.2091 - val_loss: 2.0111 - val_accuracy: 0.2108
Epoch 3/12
494/494 [==============================] - 8543s 17s/step - loss: 2.0135 - accuracy: 0.2091 - val_loss: 2.0113 - val_accuracy: 0.2108
Epoch 4/12
494/494 [==============================] - 8376s 17s/step - loss: 2.0135 - accuracy: 0.2100 - val_loss: 2.0111 - val_accuracy: 0.2108
Epoch 5/12
494/494 [==============================] - 8080s 16s/step - loss: 2.0129 - accuracy: 0.2108 - val_loss: 2.0109 - val_accuracy: 0.2108
Epoch 6/12
494/494 [==============================] - 8394s 17s/step - loss: 2.0127 - accuracy: 0.2107 - val_loss: 2.0110 - val_accuracy: 0.2108
Epoch 7/12
494/494 [==============================] - ETA: 0s - loss: 2.0120 - accuracy: 0.2107

Epoch 1/12
494/494 [==============================] - 10634s 22s/step - loss: 2.0932 - accuracy: 0.2091 - val_loss: 2.0123 - val_accuracy: 0.2108
Epoch 2/12
494/494 [==============================] - 8973s 18s/step - loss: 2.0142 - accuracy: 0.2091 - val_loss: 2.0111 - val_accuracy: 0.2108
Epoch 3/12
494/494 [==============================] - 8543s 17s/step - loss: 2.0135 - accuracy: 0.2091 - val_loss: 2.0113 - val_accuracy: 0.2108
Epoch 4/12
 31/494 [>.............................] - ETA: 1:11:07 - loss: 1.9911 - accuracy: 0.2278

for the record after 1751 min of processing and epoch 10/15

Found 15793 images belonging to 8 classes.
Found 15793 images belonging to 8 classes.
Found 0 images belonging to 0 classes.



Downloading data from https://storage.googleapis.com/tensorflow/keras-applications/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5
58889256/58889256 [==============================] - 5s 0us/step
Base model: VGG16
Number of additional layers: 4
New model name: my_new_model
New model summary: 
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 input_1 (InputLayer)        [(None, 360, 360, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 360, 360, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 360, 360, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 180, 180, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 180, 180, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 180, 180, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 90, 90, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 90, 90, 256)       295168    
                                                                 
 block3_conv2 (Conv2D)       (None, 90, 90, 256)       590080    
                                                                 
 block3_conv3 (Conv2D)       (None, 90, 90, 256)       590080    
                                                                 
 block3_pool (MaxPooling2D)  (None, 45, 45, 256)       0         
                                                                 
 block4_conv1 (Conv2D)       (None, 45, 45, 512)       1180160   
                                                                 
 block4_conv2 (Conv2D)       (None, 45, 45, 512)       2359808   
                                                                 
 block4_conv3 (Conv2D)       (None, 45, 45, 512)       2359808   
                                                                 
 block4_pool (MaxPooling2D)  (None, 22, 22, 512)       0         
                                                                 
 block5_conv1 (Conv2D)       (None, 22, 22, 512)       2359808   
                                                                 
 block5_conv2 (Conv2D)       (None, 22, 22, 512)       2359808   
                                                                 
 block5_conv3 (Conv2D)       (None, 22, 22, 512)       2359808   
                                                                 
 block5_pool (MaxPooling2D)  (None, 11, 11, 512)       0         
                                                                 
 flatten (Flatten)           (None, 61952)             0         
                                                                 
 dense (Dense)               (None, 256)               15859968  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 8)                 1032      
                                                                 
=================================================================
Total params: 30,608,584
Trainable params: 15,893,896
Non-trainable params: 14,714,688
_________________________________________________________________
Epoch 1/15
494/494 [==============================] - 20650s 42s/step - loss: 2.0458 - accuracy: 0.2454 - val_loss: 1.6292 - val_accuracy: 0.3757
Epoch 2/15
494/494 [==============================] - 18976s 38s/step - loss: 1.8339 - accuracy: 0.2815 - val_loss: 1.5704 - val_accuracy: 0.3885
Epoch 3/15
494/494 [==============================] - 10437s 21s/step - loss: 1.8078 - accuracy: 0.2937 - val_loss: 1.5843 - val_accuracy: 0.4663
Epoch 4/15
494/494 [==============================] - 8847s 18s/step - loss: 1.8024 - accuracy: 0.3033 - val_loss: 1.5287 - val_accuracy: 0.4754
Epoch 5/15
494/494 [==============================] - 8736s 18s/step - loss: 1.7824 - accuracy: 0.3200 - val_loss: 1.9303 - val_accuracy: 0.1915
Epoch 6/15
494/494 [==============================] - 8075s 16s/step - loss: 1.7628 - accuracy: 0.3320 - val_loss: 1.4775 - val_accuracy: 0.4619
Epoch 7/15
494/494 [==============================] - 8705s 18s/step - loss: 1.7693 - accuracy: 0.3281 - val_loss: 1.5611 - val_accuracy: 0.4229
Epoch 8/15
494/494 [==============================] - 9461s 19s/step - loss: 1.7657 - accuracy: 0.3292 - val_loss: 1.8903 - val_accuracy: 0.2128
Epoch 9/15
494/494 [==============================] - 9644s 20s/step - loss: 1.7561 - accuracy: 0.3327 - val_loss: 1.5375 - val_accuracy: 0.4453
Epoch 10/15
174/494 [=========>....................] - ETA: 48:56 - loss: 1.7513 - accuracy: 0.3314

In [ ]:

# evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_data)
print("Test accuracy:", test_acc)

# make predictions on the test set
y_test = np.concatenate([y for x, y in test_data], axis=0)
y_test_proba = model.predict(test_data, batch_size=50)
y_pred_test = np.argmax(y_test_proba, axis=-1)


In [ ]:

# compute the confusion matrix
confusion_mtx = confusion_matrix(y_test, y_pred_test)
print("Confusion matrix:\n", confusion_mtx)

previous standalone model for memory
# create the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(8, activation='softmax')
])

# compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()
